### --- Day 23: Unstable Diffusion ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/23

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2022, 23);
WriteLines(inputLines);

Loading puzzle file: Day23.txt
Total lines: 70
Max line length: 70

###..#####..#....##.#..#...#.#.#..#.##..##...#.#.###.#.###...###.#....
#..#..#.#....#.#.####.#.#.....#..####...####.##.#.#....####...#..#.#..
...##...#.#####..###......#.#.#..##.#..#..#####.#..###.#.####.#.####..
#.#...#...#.#..##.#.#..#.####...#.#####.###.##....##.###.#.##..####.#.
...##.##..#.##.###..##.####.##.#...##...#.#...#.##.#..##...###.###....


In [4]:
string[] testInputLines = 
[
    "....#..",
    "..###.#",
    "#...#.#",
    ".#...##",
    "#.###..",
    "##.#.##",
    ".#..#..",
];

Ok, for this one I think we can model each elf individually, noting their current position. If I understand the puzzle correctly, the move order is global so we can model that globally. Looks pretty straightforward.

In [5]:
using PointSearch = (Point stepDir, SCG.IList<Point> checks);

class ElfSimBase
{
    HashSet<Point> _elves;

    List<PointSearch> _pointSearches = [
        // North, South, West, East
        (Up, [Up + Left, Up, Up + Right]),
        (Down, [Down + Left, Down, Down + Right]),
        (Left, [Left + Up, Left, Left + Down]),
        (Right, [Right + Up, Right, Right + Down]),
    ];
    int _searchIndex = 0; // Start with North

    public ElfSimBase(string[] inputLines)
    {
        CharGrid grid = new(inputLines);
        _elves = grid.Enumerate().Where(gch => gch.ch is '#').Select(gch => gch.point).ToHashSet();
    }

    protected HashSet<Point> Elves => _elves;

    protected void MoveElves()
    {
        var q = from e in _elves
                let next = FindNextMove(e)
                group e by next into eNext
                select eNext;

        var nextElves = q.SelectMany(e => e.Count() > 1 ? e.AsEnumerable() : [e.Key]).ToHashSet();
        _elves = nextElves;
        _searchIndex += 1;
    }

    protected int CountEmptyGroundTiles()
    {
        var (xMin, xMax) = MinMax(_elves.Select(e => e.X));
        var (yMin, yMax) = MinMax(_elves.Select(e => e.Y));

        var area = (xMax - xMin + 1) * (yMax - yMin + 1);
        return area - _elves.Count();
    }

    Point FindNextMove(Point elf)
    {
        var q = from i in Range(0, 4)
                let pointSearch = _pointSearches[(_searchIndex + i) % 4]
                where pointSearch.checks.All(c => !_elves.Contains(elf + c))
                select elf + pointSearch.stepDir;
        var qList = q.ToList();

        return qList.Count is 0 or 4 ? elf : qList[0];
    }
}

In [6]:
// Spoiler alert: base class structure required to reuse state for part 2

class Part1ElfSim(string[] inputLines) : ElfSimBase(inputLines)
{
    public int PlantSeedlings()
    {
        foreach (var i in Range(0, 10))
        {
            MoveElves();
        }
        return CountEmptyGroundTiles();
    }
}

In [7]:
// In this region, the number of empty ground tiles is 110.

var testAnswer  = new Part1ElfSim(testInputLines).PlantSeedlings();
Console.WriteLine(testAnswer);

110


In [8]:
// Simulate the Elves' process and find the smallest rectangle that contains the
// Elves after 10 rounds. How many empty ground tiles does that rectangle
// contain?

var part1Answer = new Part1ElfSim(inputLines).PlantSeedlings();
Console.WriteLine(part1Answer);

3689


In [9]:
// 3689 is correct!
Ensure(3689, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/23

Ok, since we are modelling the elf positions as a `HashSet`, we can just compare one round to the next, and if there is no difference, we have found the correct round. Hopefully straightforward once again, assuming there is not millions of rounds!

In [11]:
class Part2ElfSim(string[] inputLines) : ElfSimBase(inputLines)
{
    public int CountRounds()
    {
        var previous = Elves;
        var rounds = 0;

        while (rounds < 100_000)
        {
            MoveElves();
            rounds += 1;

            if (Elves.SetEquals(previous))
            {
                return rounds;
            }

            previous = Elves;
        }

        throw new Exception("Too many rounds");
    }
}

In [12]:
// In the example above, the first round where no Elf moved was round 20:

var part2TestAnswer = new Part2ElfSim(testInputLines).CountRounds();
Console.WriteLine(part2TestAnswer);

20


In [13]:
// Figure out where the Elves need to go. What is the number of the first round
// where no Elf moves?

var part2Answer = new Part2ElfSim(inputLines).CountRounds();
Console.WriteLine(part2Answer);

965


In [14]:
// 965 is correect!
Ensure(965, part2Answer);